In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

from models.game_state import GameWorld

mini_llm = ChatOpenAI(temperature=0.7, model="gpt-4.1-mini").with_structured_output(GameWorld)
not_mini_llm = ChatOpenAI(temperature=0.7, model="gpt-4.1").with_structured_output(GameWorld)

In [3]:
import asyncio


messages = [
    SystemMessage(
        """
        You are generating a game world for a intractive narrative to take place within.
        Other LLM generations will iterate on this world, each controlling a single actor (character)
        by generating facts for them.

        The story is about is an active and escalating zombie outbreak.
        The setting is a suburban neighborhood.
        Generate at least five locations within this neighborhood.
        Generate three survivor characters that are all part of a family that lives in this neighborhood.
        Generate at least two zombie characters that are mindless and will pursue and attack any non-zombie actor they encounter
        Generate one active scenario involving all three survivors.
        Generate some history within this scenario so that we are joining in media res.
            Note that there is a game loop that cycles through all actors in a scenario round-robin style.
            So the history within a scenario should reflect this with each actor in turn having outcomes resolved and then responding
        """),
    HumanMessage("Generate a game world according to the above instructions")
]

mini_futures = [mini_llm.ainvoke(messages) for i in range(1)]
not_mini_futures = [not_mini_llm.ainvoke(messages) for i in range(1)]
mini_responses = await asyncio.gather(*mini_futures)
not_mini_responses = await asyncio.gather(*not_mini_futures)

print("GPT 4.1 MINI")
for response in mini_responses:
    print(response)
    print("##############################################")

print("##############################################")
print("##############################################")
print("##############################################")
print("GPT 4.1 (NOT MINI)")
for response in not_mini_responses:
    print(response)
    print("##############################################")

GPT 4.1 MINI
locations=[GeneratableTuple[LocationId, LocationEntity](key=LocationId(value='location_1'), value=LocationEntity(observable=ObservableLocationEntity(facts=['Suburban neighborhood street lined with detached houses'], keywords=[Keyword(value='suburban'), Keyword(value='residential'), Keyword(value='street')], id=LocationId(value='location_1'), state_history=[ObservableLocationState(facts=['Quiet residential street with some abandoned vehicles'], keywords=[Keyword(value='quiet'), Keyword(value='abandoned')], condition=<ObservableCondition.GOOD_CONDITION: 'GOOD_CONDITION'>)]), narrative_description='Maple Street is a typical suburban street with rows of detached houses, some with front lawns and cars parked in driveways. The street is eerily quiet, with signs of recent abandonment.')), GeneratableTuple[LocationId, LocationEntity](key=LocationId(value='location_2'), value=LocationEntity(observable=ObservableLocationEntity(facts=['The Thompson family house with boarded windows']

In [4]:
from devtools import pprint

pprint(not_mini_responses[0])

GameWorld(
    locations=[
        GeneratableTuple[LocationId, LocationEntity](
            key=LocationId(
                value='johnson_house',
            ),
            value=LocationEntity(
                observable=ObservableLocationEntity(
                    facts=[
                        'A two-story house with an attached garage',
                        'Front yard is overgrown, windows are boarded up',
                        'Home to the Johnson family',
                    ],
                    keywords=[
                        Keyword(
                            value='residential',
                        ),
                        Keyword(
                            value='family_home',
                        ),
                    ],
                    id=LocationId(
                        value='johnson_house',
                    ),
                    state_history=[
                        ObservableLocationState(
                            facts=[
   